## Introduction to  II (Function calling)

Below we are loading the necessary libraries, utilities, and configurations.

<a href="https://colab.research.google.com/github/adithya-s-k/AI-Engineering.academy/blob/main/PromptEngineering/function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install openai
# !pip install python-dotenv
# !pip install rich

In [9]:
import os
import openai
from rich import print
from dotenv import load_dotenv
import json

load_dotenv()

local_api_key = os.getenv("GROQ_API_KEY")
# openai.api_key = os.getenv("GROQ_API_KEY")

client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=local_api_key
)

In [3]:
def get_completion(messages, model="llama-3.2-90b-text-preview", temperature=0, max_tokens=300, tools=None):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        tools=tools
    )
    return response.choices[0].message

## Adding two numbers

In [12]:
## Define Function

def add_numbers(number_1 , number_2):
    return int(number_1) + int(number_2)
    

In [17]:
add_tool = [
    {
        "type": "function",
        "function": {
            "name": "add_numbers",
            "description": "Adds two numbers and returns the sum",
            "parameters": {
                "type": "object",
                "properties": {
                    "number_1": {
                        "type": "number",
                        "description": "The first number to add",
                    },
                    "number_2": {
                        "type": "number",
                        "description": "The second number to add",
                    },
                },
                "required": ["number_1", "number_2"],
            },
        },   
    }
]

In [14]:
messages = [
    {
        "role": "user",
        "content": "add 66034 and 39385"
    }
]

#### Without Using Function Calling

In [15]:
response = get_completion(messages)
print(response)

ChatCompletionMessage(
    content='66034 + 39385 = 105419.',
    refusal=None,
    role='assistant',
    function_call=None,
    tool_calls=None
)

#### Using Function Calling

In [18]:
response = get_completion(messages, tools=add_tool)
print(response)

ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    function_call=None,
    tool_calls=[
        ChatCompletionMessageToolCall(
            id='call_m209',
            function=Function(arguments='{"number_1": 66034, "number_2": 39385}', name='add_numbers'),
            type='function'
        )
    ]
)

In [19]:
args = json.loads(response.tool_calls[0].function.arguments)
print(add_numbers(**args))

132068

## Counting the number of "r" in strawberry

## Getting Current Weather

In [4]:
# Defines a dummy function to get the current weather
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather = {
        "location": location,
        "temperature": "50",
        "unit": unit,
    }

    return json.dumps(weather)

In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string", 
                        "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },   
    }
]

In [6]:
messages = [
    {
        "role": "user",
        "content": "What is the weather like in London?"
    }
]

In [7]:
response = get_completion(messages, tools=tools)
print(response)

ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    function_call=None,
    tool_calls=[
        ChatCompletionMessageToolCall(
            id='call_wvzq',
            function=Function(arguments='{"location": "London", "unit": "celsius"}', name='get_current_weather'),
            type='function'
        )
    ]
)

In [10]:
args = json.loads(response.tool_calls[0].function.arguments)

In [11]:
get_current_weather(**args)

'{"location": "London", "temperature": "50", "unit": "celsius"}'

## Duck Duck Go Search